In [ ]:
pip install psycopg2-binary scikit-learn

In [ ]:
pip install psycopg2-binary scikit-learn    

In [ ]:
pip install scikit-learn-extra

In [ ]:
pip install --upgrade pip


In [ ]:

pip install num

# Connect to Server

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score


# Add the project root directory to the system path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))
from data_access.postgres_handler import PostgresHandler

# Initialize the PostgresHandler
handler = PostgresHandler(
    database="nutanix",
    user="postgres",
    host='172.25.221.34',
    password="Senna",
    port=1433
)
handler.connect()

# Define columns to fetch
columns = [
    'concord_id', 'data_type', 'name', 'metric', 'queue_depth', 'num_jobs', 'blocksize','unit', 'min_measure', 
    'mean_measure', 'median_measure', 'max_measure', 'stddev_measure', 'capacity_gib', 
    'device_type', 'model', 'operating_pci_speed_gts', 'operating_pci_width', 
]

df = handler.get_data("ssd_clean_data", columns, limit=None, encode=False)
df = df[df['data_type'] == 'Sequential Write']
# Disconnect from the database
handler.disconnect()
# Filter dataset for Sequential Write

# Define numeric columns for clustering (REMOVED:  'stddev_measure', 'median_measure','min_measure','max_measure', )
numeric_columns = [
    'queue_depth', 'num_jobs', 'blocksize', 'min_measure', 
    'mean_measure', 'median_measure', 'max_measure', 'stddev_measure','operating_pci_speed_gts', 'operating_pci_width', 
]

df = pd.get_dummies(df, columns=['metric'])
print(df)



In [ ]:
# Define input columns for clustering, excluding 'mean_measure' and any other output metrics
input_columns = [
    'blocksize', 
    'num_jobs', 
    'queue_depth',
    'operating_pci_speed_gts',  
    'operating_pci_width', 
    # 'capacity_gib', 
    'metric_bw',
    'metric_iops', 
    'metric_latency'
]


# # Impute missing values in input columns
imputer = SimpleImputer(strategy="mean")
df[input_columns] = imputer.fit_transform(df[input_columns])

# Standardize input features for clustering
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[input_columns])


# Step 2: Apply PCA
pca = PCA(n_components=0.96)  # Retain 96% of the variance
principal_components = pca.fit_transform(df_scaled)

# Apply KMeans for 6 clusters
kmeans = kmeans = KMeans(
    n_clusters=8,
    init='k-means++',
    n_init=100,
    max_iter=5000,
    tol=1e-4,
    random_state=42,
    algorithm='elkan',
    verbose=0
)
df['Cluster'] = kmeans.fit_predict(df_scaled)

# Calculate silhouette score
silhouette_avg = silhouette_score(df_scaled, df['Cluster'])
print(f"Silhouette Score with 6 Clusters (excluding 'mean_measure' and other output values): {silhouette_avg}")

# Calculate mean of 'mean_measure' for each cluster
cluster_mean_measure = df.groupby('Cluster')['mean_measure'].mean()

# Sort clusters by mean_measure in descending order and display
sorted_cluster_mean_measure = cluster_mean_measure.sort_values(ascending=False)
print("Clusters ordered by 'mean_measure':")
print(sorted_cluster_mean_measure)


# Identify the highest performance cluster based on mean of 'mean_measure'
top_cluster = cluster_mean_measure.idxmax()
print(f"Top Cluster based on 'mean_measure': Tiering {top_cluster}")

# Use PCA for 2D visualization of clusters
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_scaled)

# Plot the clusters
plt.figure(figsize=(8, 6))
scatter = plt.scatter(df_pca[:, 0], df_pca[:, 1], c=df['Cluster'], cmap='viridis', s=50)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title("KMeans Clusters (excluding 'mean_measure' and output values)")
plt.colorbar(scatter, label='Cluster')
plt.show()

# Generate a heatmap of the mean values for each input feature across clusters
cluster_means = df.groupby('Cluster')[input_columns].mean()
cluster_means_normalized = (cluster_means - cluster_means.mean()) / cluster_means.std()

# Plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(cluster_means_normalized.T, annot=True, cmap="coolwarm", fmt=".2f", cbar=True, linewidths=0.5)
plt.title('Normalized Mean Values of Input Features Across Clusters')
plt.xlabel("Cluster")
plt.ylabel("Feature")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Define a range of cluster numbers to test
cluster_range = range(2, 15)
inertia_values = []
silhouette_scores = []  # Initialize silhouette_scores

# Define input columns for clustering, excluding 'mean_measure' and any other output metrics
input_columns = [
    'blocksize', 
    'num_jobs', 
    'queue_depth',
    'operating_pci_speed_gts',  
    'operating_pci_width', 
    # 'capacity_gib', 
    'metric_bw',
    'metric_iops', 
    'metric_latency'
]

# Standardize input features for clustering
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[input_columns])

# Step 2: Apply PCA
pca = PCA(n_components=0.96)  # Retain 96% of the variance
principal_components = pca.fit_transform(df_scaled)

# Loop over different numbers of clusters and compute inertia and silhouette score
for n_clusters in cluster_range:
    kmeans = KMeans(
        n_clusters=n_clusters,
        init='k-means++',
        n_init=100,
        max_iter=1000,
        random_state=42,
        algorithm='elkan'
    )
    cluster_labels = kmeans.fit_predict(principal_components)
    # inertia_values.append(kmeans.inertia_)
    
    sil_score = silhouette_score(principal_components, cluster_labels)
    silhouette_scores.append(sil_score)
    print(sil_score)

# # Plot the Elbow Graph
# plt.figure(figsize=(8, 6))
# plt.plot(cluster_range, inertia_values, marker='o', label='Inertia')
# plt.xlabel('Number of Clusters')
# plt.ylabel('Inertia')
# plt.title('Elbow Method for Optimal Number of Clusters')
# plt.legend()
# plt.show()

# Plot the Silhouette Scores
plt.figure(figsize=(8, 6))
plt.plot(cluster_range, silhouette_scores, marker='o', color='red', label='Silhouette Score')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Analysis for Optimal Number of Clusters')
plt.legend()
plt.show()


In [ ]:
pip install umap-learn

In [ ]:
pip install numpy==2.0

# Remove Outliers 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE

# Define input columns for clustering, excluding 'mean_measure' and any other output metrics
input_columns = [
    'blocksize', 
    'num_jobs', 
    'queue_depth',
    'operating_pci_speed_gts',  
    'operating_pci_width', 
    # 'capacity_gib', 
    'metric_bw',
    # 'metric_iops', 
    'metric_latency'
]

# Standardize input features for clustering
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[input_columns])

# Step 1: Detect and Remove Outliers Based on Z-Score
z_scores = np.abs((df_scaled - np.mean(df_scaled, axis=0)) / np.std(df_scaled, axis=0))  # Calculate Z-scores
outlier_threshold = 3  # Define Z-score threshold for outliers
outlier_indices = np.where(z_scores > outlier_threshold)[0]  # Find indices of outliers

# Remove outliers from the original dataframe and the scaled data
df_no_outliers = df.drop(index=outlier_indices).reset_index(drop=True)
df_scaled_no_outliers = np.delete(df_scaled, outlier_indices, axis=0)

# Print the number of outliers that were dropped
num_outliers = len(np.unique(outlier_indices))  # Get unique outlier indices and count them
print(f"Number of outliers removed: {num_outliers}")

# Step 2: Apply KMeans for clustering on the scaled, outlier-removed data
kmeans = KMeans(
    n_clusters=8,            
    init='random',        
    n_init=5000,             
    max_iter=10000,          
    tol=1e-4,                
    random_state=42,         
    algorithm='lloyd',       
    verbose=0                
)

df_no_outliers['Cluster'] = kmeans.fit_predict(df_scaled_no_outliers)

# Step 3: Calculate Silhouette Score
silhouette_avg = silhouette_score(df_scaled_no_outliers, df_no_outliers['Cluster'])
print(f"Silhouette Score with 8 Clusters (excluding outliers): {silhouette_avg}")

# Step 4: Calculate Mean of 'mean_measure' for Each Cluster
cluster_mean_measure = df_no_outliers.groupby('Cluster')['mean_measure'].mean()

# Calculate average numerical statistics for each cluster
numerical_columns = [
    'blocksize', 
    'num_jobs', 
    'queue_depth',
    'operating_pci_speed_gts',  
    'operating_pci_width'
]
cluster_numerical_stats = df_no_outliers.groupby('Cluster')[numerical_columns].mean()

# Print the average numerical statistics for each cluster
print("Average Numerical Stats for Each Cluster:")
print(cluster_numerical_stats)

# Dominance of 'metric_bw', 'metric_iops', 'metric_latency' in each cluster
dominant_features = ['metric_bw', 'metric_iops', 'metric_latency']
cluster_dominance_stats = df_no_outliers.groupby('Cluster')[dominant_features].agg(lambda x: x.mode()[0])

print("\nMost Dominant One-Hot Encoded Feature in Each Cluster:")
print(cluster_dominance_stats)

# Sort clusters by 'mean_measure' in descending order and display
sorted_cluster_mean_measure = cluster_mean_measure.sort_values(ascending=False)
print("Clusters ordered by 'mean_measure':")
print(sorted_cluster_mean_measure)

# Identify the highest performance cluster based on 'mean_measure'
top_cluster = cluster_mean_measure.idxmax()
print(f"Top Cluster based on 'mean_measure': Tiering {top_cluster}")

# Step 5: Use PCA for 2D Visualization of Clusters
pca = PCA(n_components=2)
df_pca_no_outliers = pca.fit_transform(df_scaled_no_outliers)

# Plot the clusters
plt.figure(figsize=(8, 6))
scatter = plt.scatter(df_pca_no_outliers[:, 0], df_pca_no_outliers[:, 1], c=df_no_outliers['Cluster'], cmap='viridis', s=50)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title("KMeans Clusters (excluding outliers)")
plt.colorbar(scatter, label='Cluster')
plt.show()

# Step 6: Generate a Heatmap of Mean Values for Each Feature Across Clusters
cluster_means = df_no_outliers.groupby('Cluster')[input_columns].mean()
cluster_means_normalized = (cluster_means - cluster_means.mean()) / cluster_means.std()

plt.figure(figsize=(12, 8))
sns.heatmap(cluster_means_normalized.T, annot=True, cmap="coolwarm", fmt=".2f", cbar=True, linewidths=0.5)
plt.title('Normalized Mean Values of Input Features Across Clusters')
plt.xlabel("Cluster")
plt.ylabel("Feature")
plt.show()

# Step 7: Use t-SNE for Visualization of Clusters (optional)
tsne = TSNE(n_components=2, perplexity=30, n_iter=300, random_state=42)
df_tsne_no_outliers = tsne.fit_transform(df_scaled_no_outliers)

plt.figure(figsize=(8, 6))
tsne_scatter = plt.scatter(df_tsne_no_outliers[:, 0], df_tsne_no_outliers[:, 1], c=df_no_outliers['Cluster'], cmap='viridis', s=50)
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title("t-SNE Visualization of KMeans Clusters (excluding outliers)")
plt.colorbar(tsne_scatter, label='Cluster')
plt.show()


Applied PCA (optimally reduce features) + Outliers + Duplicates 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import necessary modules
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.manifold import TSNE

# Assume 'df' is your DataFrame
# If not already loaded, you should load your data into 'df' before running this code

# Define input columns for clustering
# Get all metric dummy columns
metric_columns = [col for col in df.columns if col.startswith('metric_')]

# Define input columns
input_columns = [
    'blocksize', 
    'num_jobs', 
    'queue_depth',
    'operating_pci_speed_gts',  
    'operating_pci_width'
] + metric_columns

df = df.reset_index(drop=True)

# Step 1: Data Scaling using MinMaxScaler to normalize data
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[input_columns])

# Step 2: Refine Outlier Detection using IQR method
Q1 = np.percentile(df_scaled, 25, axis=0)
Q3 = np.percentile(df_scaled, 75, axis=0)
IQR = Q3 - Q1

# Define outlier thresholds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outlier_indices = []
for i in range(df_scaled.shape[1]):
    outlier_list_col = df.index[(df_scaled[:, i] < lower_bound[i]) | (df_scaled[:, i] > upper_bound[i])].tolist()
    outlier_indices.extend(outlier_list_col)

# Remove duplicates
outlier_indices = list(set(outlier_indices))

# Remove outliers from the original dataframe and the scaled data
df_no_outliers = df.drop(index=outlier_indices).reset_index(drop=True)
df_scaled_no_outliers = np.delete(df_scaled, outlier_indices, axis=0)

print(f"Number of outliers removed: {len(outlier_indices)}")

# Step 3: Apply PCA to reduce dimensionality while retaining 95% of variance
pca = PCA(n_components=0.95, random_state=42)
df_pca_no_outliers = pca.fit_transform(df_scaled_no_outliers)

print(f"Original number of features: {df_scaled_no_outliers.shape[1]}")
print(f"Reduced number of features after PCA: {df_pca_no_outliers.shape[1]}")

# Step 4: Optimize the number of clusters using Silhouette Analysis
from sklearn.cluster import KMeans

# To avoid heavy computation, limit the range of clusters
range_n_clusters = list(range(2, 10))
silhouette_avgs = []

for n_clusters in range_n_clusters:
    kmeans = KMeans(
        n_clusters=n_clusters,
        init='k-means++',
        n_init=10,  # Reduced n_init to lower computation
        max_iter=300,  # Default value
        random_state=42
    )
    cluster_labels = kmeans.fit_predict(df_pca_no_outliers)
    silhouette_avg = silhouette_score(df_pca_no_outliers, cluster_labels)
    silhouette_avgs.append(silhouette_avg)
    print(f"For n_clusters = {n_clusters}, the average silhouette_score is : {silhouette_avg}")

# Plot the silhouette scores
plt.figure(figsize=(8, 6))
plt.plot(range_n_clusters, silhouette_avgs, marker='o')
plt.title('Silhouette Score for Various Numbers of Clusters')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Average Silhouette Score')
plt.show()

# Choose the optimal number of clusters (the one with the highest silhouette score)
optimal_k = range_n_clusters[np.argmax(silhouette_avgs)]
print(f"Optimal number of clusters based on silhouette score: {optimal_k}")

# Step 5: Apply KMeans clustering with the optimal number of clusters
kmeans = KMeans(
    n_clusters=optimal_k,
    init='k-means++',
    n_init=10,
    max_iter=300,
    random_state=42
)
df_no_outliers['Cluster'] = kmeans.fit_predict(df_pca_no_outliers)

# Calculate silhouette score
silhouette_avg = silhouette_score(df_pca_no_outliers, df_no_outliers['Cluster'])
print(f"Silhouette Score with {optimal_k} Clusters: {silhouette_avg}")

# Step 6: Evaluate Clustering with Additional Metrics
ch_score = calinski_harabasz_score(df_pca_no_outliers, df_no_outliers['Cluster'])
db_score = davies_bouldin_score(df_pca_no_outliers, df_no_outliers['Cluster'])

print(f"Calinski-Harabasz Index: {ch_score}")
print(f"Davies-Bouldin Index: {db_score}")

# Step 7: Analyze Cluster Characteristics
# Calculate Mean of 'mean_measure' for Each Cluster
cluster_mean_measure = df_no_outliers.groupby('Cluster')['mean_measure'].mean()

# Calculate average numerical statistics for each cluster
numerical_columns = [
    'blocksize', 
    'num_jobs', 
    'queue_depth',
    'operating_pci_speed_gts',  
    'operating_pci_width'
]
cluster_numerical_stats = df_no_outliers.groupby('Cluster')[numerical_columns].mean()

# Print the average numerical statistics for each cluster
print("Average Numerical Stats for Each Cluster:")
print(cluster_numerical_stats)

# Dynamically get available metric columns
metric_columns = [col for col in df_no_outliers.columns if col.startswith('metric_')]

# Check if metric columns are available
if metric_columns:
    # Dominant features in each cluster
    dominant_features = metric_columns
    cluster_dominance_stats = df_no_outliers.groupby('Cluster')[dominant_features].agg(lambda x: x.mode().iloc[0])

    print("\nMost Dominant Features in Each Cluster:")
    print(cluster_dominance_stats)
else:
    print("\nNo metric columns available to calculate dominant features.")

# Step 8: Visualize Clusters using PCA Components
plt.figure(figsize=(8, 6))
scatter = plt.scatter(df_pca_no_outliers[:, 0], df_pca_no_outliers[:, 1], c=df_no_outliers['Cluster'], cmap='tab10', s=50)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title("KMeans Clusters Visualized with PCA Components")
plt.colorbar(scatter, label='Cluster')
plt.show()

# Step 9: Visualize Clusters using t-SNE
# Adjust t-SNE parameters to be less resource-intensive
tsne = TSNE(n_components=2, perplexity=30, n_iter=500, random_state=42)
df_tsne_no_outliers = tsne.fit_transform(df_pca_no_outliers)

plt.figure(figsize=(8, 6))
tsne_scatter = plt.scatter(df_tsne_no_outliers[:, 0], df_tsne_no_outliers[:, 1], c=df_no_outliers['Cluster'], cmap='tab10', s=50)
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title("t-SNE Visualization of KMeans Clusters")
plt.colorbar(tsne_scatter, label='Cluster')
plt.show()

# Step 10: Generate a Heatmap of Mean Values for Each Feature Across Clusters
cluster_means = df_no_outliers.groupby('Cluster')[input_columns].mean()
cluster_means_normalized = (cluster_means - cluster_means.mean()) / cluster_means.std()

plt.figure(figsize=(12, 8))
sns.heatmap(cluster_means_normalized.T, annot=True, cmap="coolwarm", fmt=".2f", cbar=True, linewidths=0.5)
plt.title('Normalized Mean Values of Input Features Across Clusters')
plt.xlabel("Cluster")
plt.ylabel("Feature")
plt.show()

# Optional: Save cluster assignments to a CSV file
# df_no_outliers.to_csv('clustered_data.csv', index=False)
 